In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
%matplotlib inline

In [2]:
pd.options.display.max_columns = None
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.float_format',lambda x: '%.4f' % x)
plt.rcParams['figure.figsize'] = 10,10

In [47]:
import folium
from folium import plugins
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

https://fred.stlouisfed.org/series/USNUM

https://www.stlouisfed.org/publications/regional-economist/second-quarter-2017/banking-deserts-become-a-concern-as-branches-dry-up

**Banking Deserts**
http://libertystreeteconomics.newyorkfed.org/2016/03/banking-deserts-branch-closings-and-soft-information.html

In [8]:
CSV_PATH = os.path.join('data', 'econ', 'US_num_of_banks.csv')
banks = pd.read_csv(CSV_PATH, index_col='DATE', parse_dates=['DATE'])

In [9]:
banks.head()

,USNUM
DATE,
1984-01-01,14400
1984-04-01,14383
1984-07-01,14389
1984-10-01,14391
1985-01-01,14381


In [13]:
CSV_PATH = os.path.join('data', 'econ', 'LSE_banking_deserts_2014.csv')
bank_locs = pd.read_csv(CSV_PATH, encoding='latin1', header=4)
bank_locs.head()

,Year,geoid2000,tract_latitude,tract_longitude,banking_desert
0,2014,1001020900,32.6444,-86.5012,1
1,2014,1001021000,32.6313,-86.7568,1
2,2014,1001021100,32.4502,-86.7156,1
3,2014,1003010100,31.1225,-87.7664,1
4,2014,1003010400,30.7260,-87.6184,1


In [60]:
us_map = folium.Map(location=[41,-97],
                    zoom_start=4,
                    tiles='Stamen Toner')

# instantiate a feature group for the incidents in the dataframe
bank_deserts = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, des in zip(bank_locs.tract_latitude, bank_locs.tract_longitude, bank_locs.banking_desert):
    bank_deserts.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=2, # define how big you want the circle markers to be
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add incidents to map
us_map.add_child(bank_deserts)